<a href="https://colab.research.google.com/github/KrishnaKewat/CV/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [13]:
import os
Root = "/content/drive/MyDrive/dataset"
os.listdir(Root)

['Actor_17',
 'Actor_11',
 'Actor_21',
 'Actor_16',
 'Actor_03',
 'Actor_19',
 'Actor_10',
 'Actor_18',
 'Actor_04',
 'Actor_20',
 'Actor_23',
 'Actor_13',
 'Actor_22',
 'Actor_14',
 'Actor_02',
 'Actor_24',
 'Actor_12',
 'Actor_05',
 'Actor_09',
 'Actor_15',
 'Actor_06',
 'Actor_07',
 'Actor_08',
 'Actor_01',
 'modelForPrediction1.sav']

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [15]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [16]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [17]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/dataset/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [18]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [19]:
x_train

array([[-4.53241882e+02,  3.28475494e+01, -1.65005627e+01, ...,
         8.45198985e-04,  6.65735279e-04,  4.27301420e-04],
       [-4.98430389e+02,  4.65623016e+01, -1.32703695e+01, ...,
         2.86638067e-04,  1.16581548e-04,  7.33235720e-05],
       [-7.00612732e+02,  6.12813530e+01,  1.21871681e+01, ...,
         6.36964660e-06,  6.53920824e-06,  8.87462465e-06],
       ...,
       [-5.75377258e+02,  7.02341766e+01, -1.14560928e+01, ...,
         1.78044764e-04,  1.13304784e-04,  3.61798120e-05],
       [-5.39380310e+02,  3.26435585e+01, -1.24525795e+01, ...,
         1.57894578e-03,  5.58187428e-04,  2.49192002e-04],
       [-6.64288269e+02,  5.55132446e+01, -6.77593136e+00, ...,
         2.68707777e-06,  2.37196764e-06,  1.77749655e-06]])

In [20]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [21]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [22]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [23]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [25]:
#Predict for the test set
y_pred=model.predict(x_test)

In [26]:
y_pred

array(['fearful', 'fearful', 'disgust', 'disgust', 'calm', 'disgust',
       'happy', 'fearful', 'calm', 'disgust', 'fearful', 'fearful',
       'fearful', 'happy', 'disgust', 'fearful', 'fearful', 'calm',
       'calm', 'disgust', 'fearful', 'calm', 'calm', 'fearful', 'calm',
       'fearful', 'disgust', 'disgust', 'happy', 'calm', 'fearful',
       'fearful', 'fearful', 'fearful', 'calm', 'fearful', 'fearful',
       'calm', 'fearful', 'disgust', 'fearful', 'disgust', 'fearful',
       'disgust', 'calm', 'fearful', 'fearful', 'disgust', 'fearful',
       'fearful', 'fearful', 'calm', 'calm', 'disgust', 'calm', 'fearful',
       'calm', 'disgust', 'calm', 'disgust', 'fearful', 'disgust',
       'fearful', 'happy', 'calm', 'happy', 'disgust', 'fearful',
       'fearful', 'disgust', 'fearful', 'disgust', 'disgust', 'disgust',
       'fearful', 'fearful', 'disgust', 'disgust', 'calm', 'disgust',
       'disgust', 'disgust', 'fearful', 'calm', 'calm', 'calm', 'calm',
       'disgust', 'ca

In [38]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*130))

Accuracy: 85.31%


In [28]:
from sklearn.metrics import accuracy_score, f1_score

In [29]:
f1_score(y_test, y_pred,average=None)

array([0.74468085, 0.60784314, 0.75968992, 0.37288136])

In [30]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,fearful,fearful
1,happy,fearful
2,disgust,disgust
3,calm,disgust
4,calm,calm
5,disgust,disgust
6,happy,happy
7,happy,fearful
8,calm,calm
9,disgust,disgust


In [31]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [32]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/dataset/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['fearful'], dtype='<U7')

In [33]:
feature

array([[-6.93497009e+02,  5.00643921e+01,  5.71450531e-01,
         1.43299656e+01,  3.33637071e+00, -2.54071975e+00,
        -4.05790949e+00, -1.07119999e+01, -7.29413939e+00,
         1.74018872e+00, -4.19064236e+00,  1.95466173e+00,
        -5.24789381e+00,  2.78142977e+00, -3.16756773e+00,
        -3.40008307e+00, -2.37803197e+00, -5.68717539e-01,
        -6.47753334e+00, -1.24320555e+00, -2.80542517e+00,
        -5.43635798e+00, -4.46875393e-01, -3.63516593e+00,
        -2.98372602e+00, -5.63902557e-01, -1.65101993e+00,
        -5.55944741e-01, -3.41018438e+00, -2.24465466e+00,
        -3.13058877e+00, -2.70089960e+00, -1.88821292e+00,
        -5.54154456e-01, -3.96459913e+00, -2.13485193e+00,
        -3.94577074e+00, -1.62457860e+00, -2.03990698e+00,
        -3.62910867e+00,  6.33751631e-01,  6.48747087e-01,
         6.21731520e-01,  6.34555459e-01,  6.60326064e-01,
         6.60485923e-01,  7.00930357e-01,  7.31556237e-01,
         7.46886790e-01,  7.23434567e-01,  7.16107428e-0